In [1]:
import getpass 
import os 
from dotenv import load_dotenv

load_dotenv()

GROK_API_KEY = os.getenv("GROK_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
HG_FACE_API_KEY = os.getenv("HG_FACE_API_KEY")

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import Chroma 
from langchain_community.embeddings import HuggingFaceEmbeddings


urls = [
       "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# print("whats up")
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)

# Adding to vector database 

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",
#                                                api_key=HG_FACE_API_KEY)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
                                  # api_key=HG_FACE_API_KEY)

vectorstore = Chroma.from_documents(
    documents = doc_splits,
    collection_name = "rag_chroma",
    embedding = embeddings
)

retriever = vectorstore.as_retriever()



USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\ibsan\AppData\Local\Temp\ipykernel_50428\3108920934.py:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
c:\Users\ibsan\Desktop\RAG_PROJECTS\LangGraph\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# LLM
# Retrievr Grader 

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq 

# Data model 
class GradeDocuments(BaseModel):
    """ Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        decription = "Document are relevant to the question, 'yes' or 'no' "
    )

# LLM with function call 
llm = ChatOllama(
    model = "llama3.2:latest",
    temperature = 0.8,
    num_predict = 256,
)

structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt 
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)


retrieved_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieved_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'
